In [ ]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
import pandas as pd

#### Lettura del dataset integrato da salvare su MongoDB

In [ ]:
df_integrate = pd.read_csv("Dataset_finale.csv")

#### Connessione a MongoDB

In [ ]:
#Connesione al Database
db=client.project
#Connessione alla collection
coll=db.Integrate

#### Creazione della struttura dei documenti da salvare 

In [ ]:
docs = []            
for pos in range(0,len(df_integrate)):
    
    docs.append( {"Artist": df_integrate['Artist'][pos], "Track": df_integrate['Track'][pos],
           "Spotify": {"Stream": df_integrate['Stream'][pos], "Url artista":df_integrate['Url_spotify'][pos],
                       "Album":df_integrate['Album'][pos],"Album Type":df_integrate['Album_type'][pos],
                       "Uri":df_integrate['Uri'][pos],"Danceability":df_integrate['Danceability'][pos],
                       "Energy":df_integrate['Energy'][pos],"Key":df_integrate['Key'][pos],
                       "Loudness":df_integrate['Loudness'][pos],"Speechiness":df_integrate['Speechiness'][pos],
                       "Acousticness":df_integrate['Acousticness'][pos],"Instrumentalness":df_integrate['Instrumentalness'][pos],
                       "Liveness":df_integrate['Liveness'][pos],"Valence":df_integrate['Valence'][pos],
                       "Tempo":df_integrate['Tempo'][pos],"Duration_ms":df_integrate['Duration_ms'][pos]},
           "Youtube": { "Views": df_integrate['Views'][pos],"Likes":df_integrate['Likes'][pos],
                       "Channel":df_integrate['Channel'][pos],"Url video":df_integrate['Url_youtube'][pos],
                       "Title":df_integrate['Title'][pos],"Description":df_integrate['Description'][pos],
                       "Licensed":df_integrate['Licensed'][pos], "Official_video": df_integrate['official_video'][pos]}})
  



#### Salvataggio dei documenti all'interno della collection

In [ ]:
coll.insert_many(docs)

#### Query 1: Trovare il numero di documenti totali

In [ ]:
n_documents=[]
for doc in coll.find():
    n_documents.append(doc)

In [ ]:
len(n_documents)

#### Query 2: Trovare il numero di documenti le cui stream sono maggiori di zero

In [ ]:
non_null_streams=[]
for doc in coll.find({"Spotify.Stream":{"$gt":0}},{"Track","Artist"}):
    non_null_streams.append(doc)
    print(doc)

In [ ]:
len(non_null_streams)

#### Query 3: Trovare il numero di documenti le cui views sono maggiori di zero tra i documenti trovati prima

In [ ]:
non_null=[]
for doc in coll.find({"Youtube.Views":{"$gt":0}},{"Track","Artist"}):
    if doc in non_null_streams:
        non_null.append(doc)
        print(doc)

#### Numero di documenti con views e stream nulli

In [ ]:
len(non_null)

In [ ]:
coll_new=db.query1

#### Query 4: Trovare i documenti che hanno un numero di views(YouTube) maggiori del numero di streams(Spotify)

In [ ]:
results=[]
for doc in coll.find({"$and":[{"$expr": {"$gt": ["$Youtube.Views","$Spotify.Stream"]}},
                              {"Youtube.Official_video":True}]},{"Track","Artist"}):
    results.append(doc)
    coll_new.insert_one(doc)
    print(doc)

#### Numero di documenti

In [ ]:
len(results)

#### Query 5: Numero di canzoni per ogni artista trovato nella precedente query

In [ ]:
for doc1 in coll_new.aggregate([{"$group":{ "_id":'$Artist',"Total":{ "$sum" : 1 } } }]):
    print(doc1)